In [291]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm as tq
import time
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import ArcGIS
df_health = pd.read_csv("healthcare_noshows.csv")

df_health



,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up,Date.diff
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,False,True,False,False,False,False,True,0
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,False,False,False,False,False,True,0
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,False,False,False,False,False,False,True,0
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,False,False,False,False,False,False,True,0
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,True,True,False,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106982,2.572134e+12,5651768,F,2016-05-03,2016-06-07,56,MARIA ORTIZ,False,False,False,False,False,True,True,35
106983,3.596266e+12,5650093,F,2016-05-03,2016-06-07,51,MARIA ORTIZ,False,False,False,False,False,True,True,35
106984,1.557663e+13,5630692,F,2016-04-27,2016-06-07,21,MARIA ORTIZ,False,False,False,False,False,True,True,41
106985,9.213493e+13,5630323,F,2016-04-27,2016-06-07,38,MARIA ORTIZ,False,False,False,False,False,True,True,41


In [292]:
df_health["Scholarship"].value_counts()

Scholarship
False    96178
True     10809
Name: count, dtype: int64

In [293]:
df_health["Alcoholism"].value_counts()

Alcoholism
False    103627
True       3360
Name: count, dtype: int64

In [294]:
%pip install tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [295]:
df_health["Age"].value_counts()

Age
1      2273
52     1746
49     1652
53     1651
56     1635
       ... 
98        6
115       5
100       4
102       2
99        1
Name: count, Length: 102, dtype: int64

In [296]:
df_health = df_health.drop(columns = ["PatientId", "AppointmentID", "ScheduledDay", "AppointmentDay", "Date.diff"])

In [297]:
df_sampled = df_health.sample(n = 200, random_state=42)

In [298]:
df_sampled["Gender"].value_counts()

Gender
F    140
M     60
Name: count, dtype: int64

In [299]:
df_sampled["Alcoholism"].value_counts()

Alcoholism
False    195
True       5
Name: count, dtype: int64

In [300]:
df_sampled["Neighbourhood"].isnull()

87067    False
95047    False
38359    False
84527    False
92581    False
         ...  
2937     False
71625    False
89624    False
6650     False
45152    False
Name: Neighbourhood, Length: 200, dtype: bool

In [301]:

location = df_sampled["Neighbourhood"]
geolocator = Nominatim(user_agent="healthdataset_geocoding")
gecoding = ArcGIS(scheme="https")
def geocoder(location):
    geo = geolocator.geocode(location)
    # for i in range(len(df_sampled.index)):
    try:
        if geo:
            """
            
            This takes the location from df_sampled["Neighbourhood]
            and converts it to latitude and longitude and returns it
            as geo.latitude and geo.longitude
            
            """

            return geo.latitude, geo.longitude
        else:
            return None, None
    except:
        return None, None

In [302]:
geocoder("MATA DA PRAIA")

(-20.2760095, -40.2939889)

In [303]:
# df_health["Hipertension"] = df_health["Hipertension"].apply(lambda x: 0 if x == "False	" else 1)


In [304]:
df_health

,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up
0,F,62,JARDIM DA PENHA,False,True,False,False,False,False,True
1,M,56,JARDIM DA PENHA,False,False,False,False,False,False,True
2,F,62,MATA DA PRAIA,False,False,False,False,False,False,True
3,F,8,PONTAL DE CAMBURI,False,False,False,False,False,False,True
4,F,56,JARDIM DA PENHA,False,True,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
106982,F,56,MARIA ORTIZ,False,False,False,False,False,True,True
106983,F,51,MARIA ORTIZ,False,False,False,False,False,True,True
106984,F,21,MARIA ORTIZ,False,False,False,False,False,True,True
106985,F,38,MARIA ORTIZ,False,False,False,False,False,True,True


In [305]:
# df_sampled["Gender"] = df_sampled["Gender"].map({"M": 0, "F": 1})
# df_sampled["Scholarship"] = df_sampled["Scholarship"].map({"False	": 0, "True	": 1})
# df_sampled["Hipertension"] = df_sampled["Hipertension"].map({"False	": 0, "True	": 1})
# df_sampled["Diabetes"] = df_sampled["Diabetes"].map({"False": 0, "True": 1})
# df_sampled["Alcoholism"] = df_sampled["Alcoholism"].map({"False": 0, "True": 1})
# df_sampled["Handcap"] = df_sampled["Handcap"].map({"False": 0, "True": 1})
# df_sampled["SMS_received"] = df_sampled["SMS_received"].map({"False": 0, "True": 1})
# df_sampled["Showed_up"] = df_sampled["Showed_up"].map({"False": 0, "True": 1})

In [306]:
def bool(type):
    bool_columns = ["Scholarship", "Hipertension", "Diabetes", "Alcoholism", "Handcap", "SMS_received", "Showed_up"]
    """
    Takes all these columns and converts it to 0's being False and 1's being True
    """
    df_sampled[bool_columns] = df_sampled[bool_columns].astype(type)


In [313]:
df_sampled

,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up,Latitude,Longitude
87067,2,66,SÃO PEDRO,0,0,0,1,0,1,0,-22.551277,-47.907236
95047,2,3,JABOUR,0,0,0,0,0,0,1,-22.880764,-43.493197
38359,2,51,ROMÃO,0,1,0,0,0,1,1,-20.312347,-40.325407
84527,2,33,UNIVERSITÁRIO,0,0,0,0,0,0,1,-1.462879,-48.440286
92581,2,54,MARIA ORTIZ,0,0,0,0,0,1,1,-20.257984,-40.297927
...,...,...,...,...,...,...,...,...,...,...,...,...
2937,2,49,BONFIM,0,1,0,1,0,0,1,NaN,NaN
71625,2,1,SANTOS REIS,0,0,0,0,0,0,1,NaN,NaN
89624,2,8,JARDIM CAMBURI,0,0,0,0,0,0,1,NaN,NaN
6650,2,21,MARIA ORTIZ,0,0,0,0,0,0,1,NaN,NaN


In [308]:
def preprocessing(df_sampled):
    df_sampled["Gender"] = df_sampled["Gender"].apply(lambda x: 1 if x == "M" else 2)

    """""
    This converts Gender to 1's and 2's depending on M or F while the for loop
    Takes the geocoder and applys it to all the rows of the smaller dataset
    """""    

    for index, row in df_sampled.iterrows():
        location = df_sampled["Neighbourhood"][index]
        lat, long = geocoder(location)
        # print(f"{location} {lat} {long}")
        df_sampled.at[index,'Latitude']= lat
        df_sampled.at[index,'Longitude']= long

    return df_sampled

In [311]:
preprocessing(df_sampled)

GeocoderTimedOut: Service timed out

In [ ]:
df_sampled["Alcoholism"].value_counts()

Alcoholism
0    123
1      2
Name: count, dtype: int64

In [ ]:
df_sampled = df_sampled.drop(columns = ["Neighbourhood", "Handcap"])

In [ ]:
df_sampled

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,Showed_up,Latitude,Longitude
87067,2,66,0,0,0,1,1,0,-22.551277,-47.907236
95047,1,3,0,0,0,0,0,1,-22.880764,-43.493197
38359,2,51,0,1,0,0,1,1,-20.312347,-40.325407
84527,2,33,0,0,0,0,0,1,-1.462879,-48.440286
92581,2,54,0,0,0,0,1,1,-20.257984,-40.297927
...,...,...,...,...,...,...,...,...,...,...
88084,1,7,0,0,0,0,0,1,-20.298286,-40.294665
36275,2,47,0,0,0,0,0,0,-20.315470,-40.353417
91013,1,69,0,0,0,0,0,1,-21.574299,-45.445198
97119,2,45,0,0,0,0,0,1,-29.826768,-50.517489


In [ ]:
# df_sampled["Latitude"] = df_sampled.apply(lambda x: "", axis = 1)
# df_sampled["Longitude"] = df_sampled.apply(lambda x: "", axis = 1)

# df_sampled["Latitude"] = 0.00
# df_sampled["Longitude"] = 0.00

In [ ]:
df_sampled

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,Showed_up,Latitude,Longitude
87067,2,66,0,0,0,1,1,0,-22.551277,-47.907236
95047,1,3,0,0,0,0,0,1,-22.880764,-43.493197
38359,2,51,0,1,0,0,1,1,-20.312347,-40.325407
84527,2,33,0,0,0,0,0,1,-1.462879,-48.440286
92581,2,54,0,0,0,0,1,1,-20.257984,-40.297927
...,...,...,...,...,...,...,...,...,...,...
88084,1,7,0,0,0,0,0,1,-20.298286,-40.294665
36275,2,47,0,0,0,0,0,0,-20.315470,-40.353417
91013,1,69,0,0,0,0,0,1,-21.574299,-45.445198
97119,2,45,0,0,0,0,0,1,-29.826768,-50.517489


In [ ]:
# location = "JARDIM DA PENHA"
# geolocator = Nominatim(user_agent="my_geocoder")
# gecoding = ArcGIS(scheme="https")
# def geocoder(location):
#     geo = geolocator.geocode(location)
#     for i in range(len(df_sampled.index)):
#         try:
#             long, lat = geocoder(df_sampled["Neighbourhood"].iloc[[i]])
#             df_sampled["Longitude"][i] = long
#             df_sampled["Latitude"][i] = lat
#             return geo.latitude, geo.longitude
#         except:
#             df_sampled["Longitude"][i] = 0
#             df_sampled["Latitude"] = 0
#             pass

In [ ]:
# location = df_sampled["Neighbourhood"]
# geolocator = Nominatim(user_agent="my_geocoder")
# gecoding = ArcGIS(scheme="https")
# def geocoder(location):
#     geo = geolocator.geocode(location)
#     if geo:
#         df_sampled["Latitude"] =  geo.latitude
#         df_sampled["Longitude"] = geo.longitude
#         return df_sampled["Latitude"], df_sampled["Longitude"]
#     else:
#         return None, None

In [ ]:
len(df_sampled.index)

125

In [ ]:
geocoder(location)

GeocoderTimedOut: Service timed out

In [ ]:
# df_sampled.to_csv("Cleaned.csv", index=False)